In [1]:
from keras.datasets import cifar10
import pandas as pd
import numpy as np
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.applications import resnet50
from keras.utils import to_categorical
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [0]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [3]:
print('Train', X_train.shape, y_train.shape)
print('Test', X_test.shape, y_test.shape)

Train (50000, 32, 32, 3) (50000, 1)
Test (10000, 32, 32, 3) (10000, 1)


In [5]:
resnet_model = resnet50.ResNet50(weights='imagenet', include_top=False, input_shape= (32,32,3))

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [0]:
# preprocess the images for feeding into the resnet model
X_train = resnet50.preprocess_input(X_train)
X_test = resnet50.preprocess_input(X_test)

In [0]:
# freeze all the parameters of the resnet model
for layer in resnet_model.layers:
    layer.trainable=False

In [0]:
# add our own layers
x = resnet_model.output
x = GlobalAveragePooling2D()(x)
preds = Dense(10, activation='softmax')(x)

In [9]:
model=Model(inputs=resnet_model.input,outputs=preds)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 16, 16, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [0]:
# one hot encode the target variables
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [11]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

W0811 15:39:51.504994 140214705231744 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [12]:
model.fit(X_train, y_train, validation_split=0.2, epochs=10, batch_size = 32)

W0811 15:41:23.562686 140214705231744 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 40000 samples, validate on 10000 samples
Epoch 1/10
40000/40000 [==============================] - 96s 2ms/step - loss: 1.8828 - acc: 0.4145 - val_loss: 2.7737 - val_acc: 0.4504
Epoch 2/10
40000/40000 [==============================] - 87s 2ms/step - loss: 1.6390 - acc: 0.4784 - val_loss: 3.2324 - val_acc: 0.4184
Epoch 3/10
40000/40000 [==============================] - 89s 2ms/step - loss: 1.6192 - acc: 0.4871 - val_loss: 3.1940 - val_acc: 0.4327
Epoch 4/10
40000/40000 [==============================] - 65s 2ms/step - loss: 1.6129 - acc: 0.4860 - val_loss: 2.9350 - val_acc: 0.4640
Epoch 5/10
40000/40000 [==============================] - 53s 1ms/step - loss: 1.6171 - acc: 0.4887 - val_loss: 2.8823 - val_acc: 0.4576
Epoch 6/10
40000/40000 [==============================] - 54s 1ms/step - loss: 1.6260 - acc: 0.4861 - val_loss: 2.8304 - val_acc: 0.4583
Epoch 7/10
40000/40000 [==============================] - 53s 1ms/step - loss: 1.6133 - acc: 0.4894 - val_loss: 3.0736 - val_acc

In [13]:
model.evaluate(X_test, y_test)

10000/10000 [==============================] - 18s 2ms/step


[3.5066941825866698, 0.4138]